# AE
## Adrian Zaręba | 320672

In [1]:
import numpy as np
import random
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error
import seaborn as sns

<style>
  h1, h2, h3, h4 {
    color: #458dc8;
  }
</style>

# Algorytm Genetyczny

### Wprowadzenie

Algorytmy genetyczne to metoda optymalizacji inspirowana procesami ewolucyjnymi zachodzącymi w przyrodzie. Są one wykorzystywane do rozwiązywania problemów, które mogą być trudne lub wręcz niemożliwe do rozwiązania za pomocą tradycyjnych metod.

### Podstawowe Elementy Algorytmu Genetycznego

#### Chromosom

Chromosom to reprezentacja jednego rozwiązania problemu. W kontekście algorytmów genetycznych może to być ciąg bitów, liczby rzeczywiste, permutacje itp. W naszym przykładzie jest to zestaw wag sieci neuronowej.

#### Populacja

Populacja to zbiór możliwych rozwiązań problemu, które są oceniane i ewoluowane w trakcie działania algorytmu. Każde rozwiązanie nazywane jest osobnikiem.

#### Funkcja Fitness

Funkcja oceny, zwana również funkcją fitness, mierzy jakość poszczególnych osobników w populacji. Celem algorytmu jest maksymalizacja lub minimalizacja tej funkcji. W naszym przypadku jest to funkcja błędu średniokwadratowego.

## Dane IRIS

In [2]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y.reshape(-1, 1))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

c:\Users\zareb\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


<style>
  h1, h2, h3, h4 {
    color: #458dc8;
  }
</style>

### 1. Funkcje Aktywacji

- `sigmoid(x)`: Funkcja sigmoidalna przekształca wartości wejściowe w zakres (0, 1), co jest użyteczne w problemach klasyfikacyjnych.
- `relu(x)`: Funkcja ReLU zwraca 0 dla wartości ujemnych i samo x dla wartości dodatnich, co pomaga w unikaniu problemu zanikania gradientu.
- `tanh(x)`: Funkcja tangens hiperboliczny przekształca wartości wejściowe w zakres (-1, 1), co jest użyteczne w różnych zastosowaniach sieci neuronowych.

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def tanh(x):
    return np.tanh(x)

<style>
  h1, h2, h3, h4 {
    color: #458dc8;
  }
</style>

### 2. Inicjalizacja Populacji

- `initialize_population(input_size, hidden_layers, output_size, population_size)`: Tworzy początkową populację losowych sieci neuronowych. Każda sieć neuronowa ma określoną liczbę warstw i neurony w każdej warstwie są inicjalizowane losowymi wagami.

In [4]:
def initialize_population(input_size, hidden_layers, output_size, population_size):
    population = []
    for _ in range(population_size):
        individual = []
        layer_sizes = [input_size] + hidden_layers + [output_size]
        for i in range(len(layer_sizes) - 1):
            weights = np.random.randn(layer_sizes[i], layer_sizes[i + 1])
            individual.append(weights)
        population.append(individual)
    return population

<style>
  h1, h2, h3, h4 {
    color: #458dc8;
  }
</style>

### 3. Przekazywanie Sygnałów przez Sieć (Forward Pass)

- `forward_pass(individual, X, activation_func)`: Przetwarza dane wejściowe przez sieć neuronową, używając wybranej funkcji aktywacji, aż do uzyskania wyników na wyjściu.


In [5]:
def forward_pass(individual, X, activation_func):
    layer_output = X
    for weights in individual:
        layer_output = activation_func(np.dot(layer_output, weights))
    return layer_output

<style>
  h1, h2, h3, h4 {
    color: #458dc8;
  }
</style>

### 4. Funkcja Fitness

- `fitness(individual, X_train, y_train, activation_func)`: Mierzy jakość osobników na podstawie średniego błędu kwadratowego między przewidywanymi a rzeczywistymi wartościami. Im mniejszy błąd, tym lepszy osobnik.

In [6]:
def fitness(individual, X_train, y_train, activation_func):
    predictions = forward_pass(individual, X_train, activation_func)
    error = np.mean((y_train - predictions) ** 2)
    return -error

<style>
  h1, h2, h3, h4 {
    color: #458dc8;
  }
</style>

### 5. Selekcja

- `select(population, fitnesses)`: Wybiera najlepszych osobników z populacji do dalszej reprodukcji. Osobniki z najlepszymi wynikami fitness mają większe szanse na wybór, co zapewnia, że korzystne cechy są przekazywane potomkom.

In [7]:
def select(population, fitnesses):
    sorted_indices = np.argsort(fitnesses)
    top_half = sorted_indices[len(sorted_indices) // 2:]
    selected_indices = np.random.choice(top_half, size=len(population), replace=True)
    return [population[i] for i in selected_indices]

<style>
  h1, h2, h3, h4 {
    color: #458dc8;
  }
</style>

### 6. Krzyżowanie (Crossover)

- `crossover(parent1, parent2)`: Łączy dwa chromosomy rodzicielskie, tworząc nowe chromosomy potomne. Proces ten polega na wymianie części wag między dwa rodzicami, co prowadzi do powstania nowych kombinacji wag w potomkach.


In [8]:
def crossover(parent1, parent2):
    child1, child2 = [], []
    for w1, w2 in zip(parent1, parent2):
        point = random.randint(1, w1.size - 1)
        w1_flat, w2_flat = w1.flatten(), w2.flatten()
        c1_flat = np.concatenate((w1_flat[:point], w2_flat[point:]))
        c2_flat = np.concatenate((w2_flat[:point], w1_flat[point:]))
        child1.append(c1_flat.reshape(w1.shape))
        child2.append(c2_flat.reshape(w2.shape))
    return child1, child2

<style>
  h1, h2, h3, h4 {
    color: #458dc8;
  }
</style>
### 7. Mutacja

- `mutate(individual, mutation_rate)`: Wprowadza losowe zmiany do wag osobników, co jest ważne, aby zachować różnorodność genetyczną populacji i zapobiegać zbieżności do lokalnych minimów.


In [9]:
def mutate(individual, mutation_rate=0.01):
    for weights in individual:
        if random.random() < mutation_rate:
            mutation = np.random.randn(*weights.shape) * mutation_rate
            weights += mutation
    return individual

<style>
  h1, h2, h3, h4 {
    color: #458dc8;
  }
</style>

### 8. Algorytm Genetyczny

- `genetic_algorithm(input_size, hidden_layers, output_size, population_size, generations, activation_func)`: Łączy wszystkie kroki w jednym algorytmie genetycznym, który iteracyjnie poprawia populację sieci neuronowych. Algorytm działa przez określoną liczbę generacji, w każdej z nich przeprowadzając selekcję, krzyżowanie i mutację.

In [10]:
def genetic_algorithm(input_size, hidden_layers, output_size, population_size, generations, activation_func):
    population = initialize_population(input_size, hidden_layers, output_size, population_size)
    for generation in range(generations):
        fitnesses = np.array([fitness(ind, X_train, y_train, activation_func) for ind in population])
        population = select(population, fitnesses)
        next_population = []
        for i in range(population_size // 2):
            parent1, parent2 = population[2*i], population[2*i + 1]
            child1, child2 = crossover(parent1, parent2)
            next_population.append(mutate(child1))
            next_population.append(mutate(child2))
        population = next_population
        if generation % 10 == 0:
            best_fitness = max(fitnesses)
            print(f'Generation {generation}: Best Fitness = {1+best_fitness:.4f}', end='\r')
    
    best_individual = population[np.argmax([fitness(ind, X_train, y_train, activation_func) for ind in population])]
    return best_individual

<style>
  h1, h2, h3, h4 {
    color: #458dc8;
  }
</style>

## Zastosowanie na Zbiorze Danych Iris

In [17]:
# Pobieranie danych do odpowiedniego formatu
scaler = StandardScaler()
X_train_IRIS = scaler.fit_transform(X_train)
X_test_IRIS = scaler.transform(X_test)

# Definiowanie architekruty 
input_size = X_train_IRIS.shape[1]
hidden_layers = [5, 5]
output_size = y_train.shape[1]
population_size = 400
generations = 800
activation_func = sigmoid

# Algorytm genetyczny
best_individual = genetic_algorithm(input_size, hidden_layers, output_size, population_size, generations, activation_func)

predictions = forward_pass(best_individual, X_test_IRIS, activation_func)
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))

print(f'\nAccuracy on test set: {accuracy * 100:.2f}%')

Generation 790: Best Fitness = 0.8906
Accuracy on test set: 80.00%


<style>
  h1, h2, h3, h4 {
    color: #458dc8;
  }
</style>

## Podsumowanie

W trakcie działania algorytmu genetycznego na zbiorze danych Iris, algorytm iteracyjnie poprawiał populację sieci neuronowych, aby znaleźć optymalne wagi, które minimalizują błąd predykcji. Proces ten obejmował inicjalizację populacji, ocenę osobników za pomocą funkcji fitness, selekcję najlepszych osobników, krzyżowanie w celu wymiany informacji genetycznej oraz mutację dla zachowania różnorodności.

### Dlaczego Algorytmy Genetyczne Działają w Sieciach Neuronowych (MLP)

W sieciach neuronowych typu MLP proces uczenia polega na dostosowywaniu wag, aby minimalizować różnicę między przewidywaniami sieci a rzeczywistymi wynikami. Tradycyjne metody, takie jak algorytm wstecznej propagacji (backpropagation), mogą być skuteczne, ale czasami zmagają się z problemami, takimi jak utkwienie w lokalnych minimach. Algorytmy genetyczne, dzięki ich globalnemu podejściu do przeszukiwania przestrzeni rozwiązań, mogą uniknąć takich pułapek i znaleźć bardziej optymalne rozwiązania.

Algorytmy genetyczne działają dobrze w MLP, ponieważ:

1. **Różnorodność Rozwiązań**: Przez inicjalizację populacji losowych wag i stosowanie krzyżowania oraz mutacji, algorytmy genetyczne mogą eksplorować szeroki zakres możliwych rozwiązań.
2. **Globalne Poszukiwanie**: Algorytmy te są mniej podatne na utkwienie w lokalnych minimach w porównaniu do algorytmów gradientowych, ponieważ przeszukują przestrzeń rozwiązań bardziej globalnie.
3. **Eliminacja Złożonych Obliczeń Gradientowych**: Algorytmy genetyczne nie wymagają obliczania gradientów, co może być zaletą w przypadku skomplikowanych i niegładkich funkcji kosztu.

### Zastosowania Algorytmów Genetycznych

Oprócz optymalizacji wag w sieciach neuronowych, algorytmy genetyczne mają szerokie zastosowania w różnych dziedzinach:

- **Inżynieria**: Optymalizacja konstrukcji, projektowanie układów mechanicznych i elektronicznych.
- **Ekonomia i Finanse**: Optymalizacja portfela inwestycyjnego, prognozowanie ekonomiczne.
- **Biologia i Medycyna**: Modelowanie procesów biologicznych, analiza sekwencji DNA, projektowanie leków.
- **Informatyka**: Rozwiązywanie problemów NP-trudnych, takich jak problem komiwojażera, projektowanie układów cyfrowych.
- **Robotyka**: Optymalizacja trajektorii ruchu, projektowanie systemów kontrolnych.

Algorytmy genetyczne są wszechstronnym narzędziem, które może być dostosowane do szerokiego zakresu problemów optymalizacyjnych, dzięki swojej zdolności do eksploracji i eksploatacji dużych przestrzeni rozwiązań. W połączeniu z innymi metodami uczenia maszynowego i sztucznej inteligencji, mogą prowadzić do bardziej efektywnych i innowacyjnych rozwiązań w wielu dziedzinach.